In [2]:
#!pip install PyTsetlinMachineCUDA
#!pip install pycuda
import numpy as np
from time import time
from PyTsetlinMachineCUDA.tm import MultiClassConvolutionalTsetlinMachine2D

## Fashion MNIST

In [3]:
# load fashion mnist
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

2.3.0
4423680/4422102 [==============================] - 0s 0us/step


In [4]:
#binarizing
import numpy as np
import cv2
y_train = train_labels
y_test = test_labels

X_train = np.copy(train_images)
X_test = np.copy(test_images)


for i in range(X_train.shape[0]): 
  X_train[i,:] = cv2.adaptiveThreshold(X_train[i], 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

for i in range(X_test.shape[0]): 
  X_test[i,:] = cv2.adaptiveThreshold(X_test[i], 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)


In [5]:
np.unique(X_train), np.unique(X_test)

(array([0, 1], dtype=uint8), array([0, 1], dtype=uint8))

In [6]:
#using Tsetlin Machine on Fashion MNIST, https://github.com/cair/PyTsetlinMachineCUDA/blob/master/PyTsetlinMachineCUDA/tm.py

tm = MultiClassConvolutionalTsetlinMachine2D(1000, 100*100, 10.0, (20, 20), max_weight=255)
"""
number_of_clauses=8000
T=100*100
s=10.0
patch_dim=(10, 10),
"""

for i in range(10):
  start_train = time()
  tm.fit(X_train, y_train, epochs=1, incremental=True)
  stop_train = time()

  test_accuracy = (tm.predict(X_test) == y_test).mean()
  train_accuracy = (tm.predict(X_train) == y_train).mean()

  print('Epoch', i+1, ', Train Accuracy: ', train_accuracy,', Test Accuracy: ', test_accuracy, ', Train Time: ', stop_train - start_train)
  

Epoch 1 , Train Accuracy:  0.7452833333333333 , Test Accuracy:  0.7391 , Train Time:  36.43207573890686
Epoch 2 , Train Accuracy:  0.7682 , Test Accuracy:  0.7615 , Train Time:  40.01711297035217
Epoch 3 , Train Accuracy:  0.81295 , Test Accuracy:  0.8063 , Train Time:  41.19944787025452
Epoch 4 , Train Accuracy:  0.83775 , Test Accuracy:  0.8257 , Train Time:  40.974682569503784
Epoch 5 , Train Accuracy:  0.8499333333333333 , Test Accuracy:  0.8374 , Train Time:  40.882296323776245
Epoch 6 , Train Accuracy:  0.8612333333333333 , Test Accuracy:  0.843 , Train Time:  41.11309766769409
Epoch 7 , Train Accuracy:  0.8663166666666666 , Test Accuracy:  0.8511 , Train Time:  40.73964166641235
Epoch 8 , Train Accuracy:  0.8723166666666666 , Test Accuracy:  0.8516 , Train Time:  40.64657926559448
Epoch 9 , Train Accuracy:  0.87745 , Test Accuracy:  0.8568 , Train Time:  40.92972207069397
Epoch 10 , Train Accuracy:  0.8799 , Test Accuracy:  0.8591 , Train Time:  40.74044322967529


### ConvNet

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(60000, 28, 28, 1)
#train_images = train_images / 255.0 #make images to 0:1
test_images = test_images.reshape(10000, 28, 28, 1)
#test_images = test_images/255.0
mean_vals = np.mean(train_images, axis=0)
std_val = np.std(train_images)

train_images = (train_images - mean_vals)/std_val
test_images = (test_images - mean_vals)/std_val

In [ ]:
from tensorflow.keras import layers, regularizers, initializers, optimizers, models

optimizer = optimizers.Adam(lr=0.0004)


# weights initilizer with seed

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # 128 number of input data
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, verbose=1, validation_data=(test_images, test_labels), shuffle=True)

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5052 - accuracy: 0.8193 - val_loss: 0.3976 - val_accuracy: 0.8590
Epoch 2/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3436 - accuracy: 0.8773 - val_loss: 0.3450 - val_accuracy: 0.8724
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3011 - accuracy: 0.8903 - val_loss: 0.3244 - val_accuracy: 0.8855
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2712 - accuracy: 0.9015 - val_loss: 0.2964 - val_accuracy: 0.8928
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2491 - accuracy: 0.9078 - val_loss: 0.2838 - val_accuracy: 0.8960
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2303 - accuracy: 0.9156 - val_loss: 0.2840 - val_accuracy: 0.8985
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2134 - accuracy: 0.9195 - val_loss: 0.2700 - val_accuracy:

## MNIST Data

In [7]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
# binarizing
X_train = np.where(X_train >= 75, 1, 0) 
X_test = np.where(X_test >= 75, 1, 0) 

tm = MultiClassConvolutionalTsetlinMachine2D(2000, 50*15, 5.0, (10, 10), max_weight=16)

for i in range(10):
    start_training = time()
    tm.fit(X_train, y_train, epochs=1, incremental=True)
    stop_training = time()

    test_accuracy = (tm.predict(X_test) ==  y_test).mean()
    train_accuracy = (tm.predict(X_train) == y_train).mean()

    print("Epoch", i+1, ", Train: ", train_accuracy,", Test: ", test_accuracy,", Time: ", stop_training-start_training)

Epoch 1 , Train:  0.9756 , Test:  0.9743 , Time:  50.00888109207153
Epoch 2 , Train:  0.9856166666666667 , Test:  0.9846 , Time:  46.062145471572876
Epoch 3 , Train:  0.98765 , Test:  0.985 , Time:  46.00924611091614
Epoch 4 , Train:  0.9889666666666667 , Test:  0.9854 , Time:  45.8051860332489
Epoch 5 , Train:  0.9909666666666667 , Test:  0.9864 , Time:  45.22726917266846
Epoch 6 , Train:  0.9913 , Test:  0.9874 , Time:  44.924257040023804
Epoch 7 , Train:  0.9925166666666667 , Test:  0.9877 , Time:  44.702162742614746
Epoch 8 , Train:  0.9926666666666667 , Test:  0.9886 , Time:  45.077603816986084
Epoch 9 , Train:  0.9930833333333333 , Test:  0.9884 , Time:  44.6230833530426
Epoch 10 , Train:  0.9926333333333334 , Test:  0.9877 , Time:  44.48716926574707


### ConvNet

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test  = X_test.reshape((X_test.shape[0], 28, 28, 1))
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train = (X_train - mean_vals)/std_val
X_test = (X_test - mean_vals)/std_val

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import layers, regularizers, initializers, optimizers, models


model = Sequential([
                    Conv2D(filters=32, kernel_size=[5,5], padding='valid', activation='relu', input_shape=(X_train.shape[1:])),
                    MaxPooling2D(pool_size=[2,2], padding='same'),
                    Conv2D(filters=64, kernel_size=[5,5], padding='valid', activation='relu'),
                    MaxPooling2D(pool_size=[2,2], padding='same'),
                    Flatten(),
                    Dense(1024, activation='relu'),
                    Dropout(.5),
                    Dense(10, activation='softmax')
                    ])

In [ ]:
optimizer = optimizers.Adam(lr=0.0004)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, shuffle=True, validation_data=(X_test, y_test))

Epoch 1/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1665 - accuracy: 0.9482 - val_loss: 0.0484 - val_accuracy: 0.9857
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0532 - accuracy: 0.9834 - val_loss: 0.0383 - val_accuracy: 0.9873
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0390 - accuracy: 0.9879 - val_loss: 0.0329 - val_accuracy: 0.9904
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0292 - accuracy: 0.9907 - val_loss: 0.0307 - val_accuracy: 0.9901
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0240 - accuracy: 0.9922 - val_loss: 0.0301 - val_accuracy: 0.9911
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0194 - accuracy: 0.9933 - val_loss: 0.0267 - val_accuracy: 0.9915
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0155 - accuracy: 0.9948 - val_loss: 0.0300 - val_accuracy: 0.9901
Epoch 

References:

https://github.com/cair